In [1]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
df = pd.read_csv('../data/raw.csv')
df = df.drop_duplicates(subset=['ticker', 'text'])
df

,ticker,text,lang,in_reply_to,is_quote_status,retweeted_tweet,possibly_sensitive,favorited,date,quote_count,...,view_count,view_count_state,retweet_count,is_translatable,edits_remaining,has_card,thumbnail_title,urls,hashtags,user
0,AAPL,BREAKING: Apple $AAPL announces its new iPhone...,en,NaN,False,NaN,0.0,False,2025-02-19 16:07:59+00:00,84.0,...,632637.0,EnabledWithCount,176.0,0.0,5.0,False,NaN,[],[],NaN
1,AAPL,BREAKING: George Soros (Soros Fund Management)...,en,NaN,False,NaN,0.0,False,2025-02-14 22:17:11+00:00,8.0,...,134905.0,EnabledWithCount,58.0,0.0,5.0,False,NaN,[],[],NaN
2,AAPL,"BREAKING: Apple, $AAPL, will restore TikTok to...",en,NaN,False,NaN,NaN,False,2025-02-14 00:27:38+00:00,49.0,...,413717.0,EnabledWithCount,195.0,0.0,5.0,False,NaN,[],[],NaN
3,AAPL,"Apple, $AAPL, will integrate Alibaba's, $BABA,...",en,NaN,False,NaN,NaN,False,2025-02-13 14:43:52+00:00,12.0,...,160997.0,EnabledWithCount,63.0,0.0,5.0,False,NaN,[],[],NaN
4,AAPL,"Apple, $AAPL, is reportedly exploring both hum...",en,NaN,False,NaN,NaN,False,2025-02-12 23:26:34+00:00,12.0,...,151919.0,EnabledWithCount,66.0,0.0,5.0,False,NaN,[],[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12067,SBUX,Guest Alert - @CNBCClosingBell 4:15pm ET - How...,en,NaN,False,NaN,NaN,False,2010-11-04 20:00:55+00:00,0.0,...,NaN,Enabled,0.0,0.0,5.0,False,NaN,[],"['earnings', 'coffee', 'commodities', 'investi...",CNBC
12068,SBUX,Look Ahead: Fed Move Promises More Cheap Dolla...,en,NaN,False,NaN,NaN,False,2010-11-03 22:13:27+00:00,0.0,...,NaN,Enabled,2.0,0.0,5.0,False,NaN,[],['Congress'],CNBC
12069,SBUX,RT @mb_willoughby @Tr8r_x $mips big earnings g...,en,28786240892.0,False,NaN,NaN,False,2010-10-26 13:38:43+00:00,0.0,...,NaN,Enabled,0.0,0.0,5.0,False,NaN,[],['overcaffienated'],Voltaire
12070,SBUX,"Video: Daily Dividend Report: SBUX, NI, BDN, M...",en,NaN,False,NaN,NaN,False,2010-03-24 21:25:08+00:00,0.0,...,NaN,Enabled,0.0,0.0,5.0,False,NaN,[],[],"Morningstar, Inc."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11453 entries, 0 to 12071
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ticker              11453 non-null  object 
 1   text                11453 non-null  object 
 2   lang                11451 non-null  object 
 3   in_reply_to         1595 non-null   object 
 4   is_quote_status     11451 non-null  object 
 5   retweeted_tweet     0 non-null      float64
 6   possibly_sensitive  6737 non-null   object 
 7   favorited           11452 non-null  object 
 8   date                11452 non-null  object 
 9   quote_count         11452 non-null  float64
 10  reply_count         11452 non-null  float64
 11  favorite_count      11451 non-null  float64
 12  view_count          2678 non-null   object 
 13  view_count_state    11452 non-null  object 
 14  retweet_count       11452 non-null  float64
 15  is_translatable     11451 non-null  float64
 16  edits_rem

# Добавление котировок

In [4]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset='date')

In [5]:
df.date.describe()

count                                  11451
mean     2018-03-06 17:31:51.698192384+00:00
min                2009-06-17 23:46:05+00:00
25%         2014-05-06 09:57:58.500000+00:00
50%                2017-09-15 14:44:51+00:00
75%         2022-10-18 18:55:08.500000+00:00
max                2025-02-24 14:51:50+00:00
Name: date, dtype: object

In [6]:
qs = pd.read_csv('../data/quotes.csv').drop(['Unnamed: 0', 'id'], axis=1).rename({'quote_date': 'date'}, axis=1)
qs

,ticker,date,open,high,low,close,volume
0,AAPL,2024-10-18,236.18,236.18,234.01,235.00,46431472
1,AAPL,2024-10-17,233.43,233.85,230.52,232.15,32993810
2,AAPL,2024-10-16,231.60,232.12,229.84,231.78,34082240
3,AAPL,2024-10-15,233.61,237.49,232.37,233.85,64751367
4,AAPL,2024-10-14,228.70,231.73,228.60,231.30,39882085
...,...,...,...,...,...,...,...
313654,SPY,2014-01-08,183.45,183.83,182.89,183.52,96582234
313655,SPY,2014-01-07,183.09,183.79,182.95,183.48,86144169
313656,SPY,2014-01-06,183.49,183.56,182.08,182.36,108028139
313657,SPY,2014-01-03,183.23,183.60,182.63,182.89,81390502


In [7]:
qs['date'] = pd.to_datetime(qs['date'])
qs = qs.drop(['high', 'low', 'volume'], axis=1)

In [8]:
qs.date.describe()

count                           313659
mean     2019-06-21 04:20:46.586898176
min                2014-01-02 00:00:00
25%                2016-10-11 00:00:00
50%                2019-07-05 00:00:00
75%                2022-03-03 00:00:00
max                2024-11-01 00:00:00
Name: date, dtype: object

In [9]:
def open_price(X, Q):
    '''
    Возвращает модифицированный датасет X c новыми 
    аттрибутами quote_date и open, полученными из
    датасета с котировками Q.
    
    open - цена открытия в день публикации или в 
    первый доступный день до этой даты
    
    quote_date - дата котировки 
    '''
    def func(x):
        '''
        Возвращает котировку, соответвующую 
        дате и тикеру из x. Если такой котировки
        нет, возвращается значение None
        '''
        current_date = str(x['date'].date())
        ticker_name = x['ticker']
        filtered = Q[(Q['date'] <= current_date) & (Q['ticker'] == ticker_name)].sort_values(by='date', ascending=False)
        if filtered.empty:
            return None
        return filtered.iloc[0]
        
    buf = X.apply(func, axis=1).rename({'date': 'quote_date',
                                        'open': 'open_last'},
                                       axis=1).drop(['close', 'ticker'], axis=1)
    
    return pd.concat([X, buf], axis=1)

df = open_price(df, qs)

In [10]:
df = df.dropna(subset='open_last')

In [11]:
def close_price(X, Q, days = 1):
    '''
    Возвращает модифицированный датасет X c новым 
    аттрибутом close_{days}_day_after, полученными из
    датасета с котировками Q. Этот атрибут содержит цену закрытия
    через days дней после публикации твита (отсчитывается от даты 
    последней котировки).
    '''
    def func(x):
        '''
        Возвращает котировку, соответвующую дате
        через days дней после даты последней котировки и тикеру из x. 
        Если такой котировки нет, возвращается значение None
        '''
        current_date = x['quote_date'].date()
        ticker_name = x['ticker']
        next_date = current_date + pd.to_timedelta(days, unit='d')
        filtered = Q[(Q['date'] >= str(next_date)) & (Q['ticker'] == ticker_name)].sort_values(by='date')
        if filtered.empty:
            return None
        return filtered.iloc[0]['close']
    
    X[f'close_{days}_day_after'] = X.apply(func, axis=1)
    
    return X

df = close_price(df, qs, 1)

In [12]:
df = df.dropna(subset='close_1_day_after')

In [13]:
df

,ticker,text,lang,in_reply_to,is_quote_status,retweeted_tweet,possibly_sensitive,favorited,date,quote_count,...,is_translatable,edits_remaining,has_card,thumbnail_title,urls,hashtags,user,quote_date,open_last,close_1_day_after
75,AAPL,$META Zuckerberg has said that they would be t...,en,NaN,False,NaN,0.0,False,2024-10-05 14:01:00+00:00,64.0,...,0.0,5.0,False,NaN,[],[],NaN,2024-10-04,227.90,221.69
76,AAPL,"You can follow all market news, including for ...",en,1.8398378693992573e+18,False,NaN,0.0,False,2024-09-28 04:40:04+00:00,0.0,...,0.0,5.0,False,NaN,[{'display_url': 'unusualwhales.com/news-feed'...,[],NaN,2024-09-27,228.46,233.00
77,AAPL,JUST IN: Apple $AAPL is now reportedly no long...,en,NaN,False,NaN,NaN,False,2024-09-28 01:21:45+00:00,92.0,...,0.0,5.0,False,NaN,[],[],NaN,2024-09-27,228.46,233.00
78,AAPL,"Apple, $AAPL, is no longer a ‘growth engine’ f...",en,NaN,False,NaN,NaN,False,2024-09-26 11:57:00+00:00,6.0,...,0.0,5.0,False,NaN,[],[],NaN,2024-09-26,227.30,227.79
79,AAPL,Here are the hottest and most active bullish a...,en,NaN,False,NaN,0.0,False,2024-09-23 18:53:35+00:00,1.0,...,0.0,5.0,False,NaN,[{'display_url': 'unusualwhales.com/hottest-co...,[],NaN,2024-09-23,227.34,227.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11941,SBUX,Probably time to go overweight $SBUX on the GS...,en,NaN,False,NaN,NaN,False,2014-01-21 20:18:11+00:00,0.0,...,0.0,5.0,False,NaN,[],[],DumbLuckCapital,2014-01-21,75.05,73.60
11942,SBUX,Are there problems brewing at Wall Street's co...,en,NaN,False,NaN,0.0,False,2014-01-14 23:10:46+00:00,0.0,...,0.0,5.0,False,NaN,"[{'display_url': 'cnb.cx/19s0hUh', 'expanded_u...",[],CNBC,2014-01-14,75.12,76.19
11943,SBUX,Get @HedgeyeHWP on that #NRNPowerList !! $DRI...,en,NaN,False,NaN,NaN,False,2014-01-13 17:22:02+00:00,0.0,...,0.0,5.0,False,NaN,[],['NRNPowerList'],"Fernando Valle, CFA",2014-01-13,77.38,75.46
11944,SBUX,Serious question: who needs who more... White ...,en,NaN,False,NaN,NaN,False,2014-01-04 15:21:52+00:00,0.0,...,0.0,5.0,False,NaN,[],[],DumbLuckCapital,2014-01-03,77.48,76.17


# Создание таргета

In [14]:
class СlsTargetTransformer(BaseEstimator, TransformerMixin):
    '''
    Класс трансформера, создающего таргет для задачи классификации.
    Положительная категория задается через процент минимальной прибыли.
    '''
    def __init__(self, shift_percent=0):
        """
        Инициализация трансформера с гиперпараметрами.
        :param shift_percent: Порог положительной категории таргета в процентах (опционально).
        """
        self.shift_percent = shift_percent

    def transform(self, X):
        """
        Метод transform: применяет преобразование к данным.
        :param X: Входные данные.
        :return: Преобразованные данные.
        """
        X_copy = X.copy()

        # for k in [1, 3, 7]:
        for k in [1]:
            col_name = f'{k}_day_after'
            X_copy[col_name] = (X_copy[f'close_{k}_day_after'] / X_copy['open_last'] - 1) * 100 - self.shift_percent
            X_copy[col_name] = X_copy[col_name].apply(lambda x: 1 if x >= 0 else 0)

        # X_copy = X_copy.drop(['close_1_day_after', 'close_3_day_after', 'close_7_day_after', 'open_last'], axis=1)
        X_copy = X_copy.drop(['close_1_day_after', 'open_last'], axis=1)

        return X_copy

In [15]:
class_target_transformer = СlsTargetTransformer()

df = class_target_transformer.transform(df)

df.head()

,ticker,text,lang,in_reply_to,is_quote_status,retweeted_tweet,possibly_sensitive,favorited,date,quote_count,...,retweet_count,is_translatable,edits_remaining,has_card,thumbnail_title,urls,hashtags,user,quote_date,1_day_after
75,AAPL,$META Zuckerberg has said that they would be t...,en,NaN,False,NaN,0.0,False,2024-10-05 14:01:00+00:00,64.0,...,143.0,0.0,5.0,False,NaN,[],[],NaN,2024-10-04,0
76,AAPL,"You can follow all market news, including for ...",en,1.8398378693992573e+18,False,NaN,0.0,False,2024-09-28 04:40:04+00:00,0.0,...,0.0,0.0,5.0,False,NaN,[{'display_url': 'unusualwhales.com/news-feed'...,[],NaN,2024-09-27,1
77,AAPL,JUST IN: Apple $AAPL is now reportedly no long...,en,NaN,False,NaN,NaN,False,2024-09-28 01:21:45+00:00,92.0,...,403.0,0.0,5.0,False,NaN,[],[],NaN,2024-09-27,1
78,AAPL,"Apple, $AAPL, is no longer a ‘growth engine’ f...",en,NaN,False,NaN,NaN,False,2024-09-26 11:57:00+00:00,6.0,...,26.0,0.0,5.0,False,NaN,[],[],NaN,2024-09-26,1
79,AAPL,Here are the hottest and most active bullish a...,en,NaN,False,NaN,0.0,False,2024-09-23 18:53:35+00:00,1.0,...,19.0,0.0,5.0,False,NaN,[{'display_url': 'unusualwhales.com/hottest-co...,[],NaN,2024-09-23,1


In [16]:
df = df.drop('quote_date', axis=1)

In [17]:
df.to_csv('../data/preprocessed.csv', index=False, encoding="utf-8")